In [1]:
import pandas as pd
from pyspark.sql import SparkSession

### Augment data for benchmarking purposes
Provided data was too low in volume. Multiply existing data into millions of row to clearly see the significant difference between reading,processing data using Spark.

In [2]:
df = pd.read_csv("../data/creditcard.csv")
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [3]:
len(df)

284807

Sampling the data into 20M Rows

In [9]:
df_20M = df.sample(20000000, replace=True)
df_20M.to_csv("../data/creditfraud_20M.csv", index=False)

### Spark Reading CSV
Build a session to read a csv file with Spark

In [2]:
%env KUBECONFIG=/workspace/kube/config

env: KUBECONFIG=/workspace/kube/config


In [3]:
spark = SparkSession.builder \
    .appName("SparkK8sRead") \
    .master("k8s://https://192.168.1.75:6443") \
    .config("spark.executor.instances", 2) \
    .config("spark.kubernetes.container.image", "spark:k8s-spark") \
    .config("spark.kubernetes.container.image.pullPolicy", "IfNotPresent") \
    .config("spark.kubernetes.authenticate.driver.serviceAccountName", "spark") \
    .config("spark.kubernetes.executor.request.cores", "1") \
    .config("spark.kubernetes.executor.limit.cores", "1") \
    .config("spark.kubernetes.executor.memory", "2g") \
    .getOrCreate()

df_spark_20m = spark.read.csv("../data/creditfraud_20M.csv", header=True, inferSchema=True)
df_spark_20m.head()

24/06/24 08:42:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/24 08:42:26 WARN ExecutorPodsAllocator: 1 new failed executors.
24/06/24 08:42:27 WARN ExecutorPodsAllocator: 1 new failed executors.
24/06/24 08:42:39 WARN ExecutorPodsAllocator: 1 new failed executors.
24/06/24 08:42:41 WARN ExecutorPodsAllocator: 1 new failed executors.
24/06/24 08:42:52 WARN ExecutorPodsAllocator: 1 new failed executors.0 + 0) / 1]
24/06/24 08:42:52 ERROR ExecutorPodsAllocator: Max number of executor failures (4) reached


Py4JJavaError: An error occurred while calling o42.csv.
: org.apache.spark.SparkException: Job 0 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1(DAGScheduler.scala:1253)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$cleanUpAfterSchedulerStop$1$adapted(DAGScheduler.scala:1251)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:79)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:1251)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:3087)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:84)
	at org.apache.spark.scheduler.DAGScheduler.$anonfun$stop$3(DAGScheduler.scala:2973)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1375)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:2973)
	at org.apache.spark.SparkContext.$anonfun$stop$12(SparkContext.scala:2263)
	at org.apache.spark.

24/06/24 08:43:03 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/06/24 08:43:12 WARN ExecutorPodsWatchSnapshotSource: Kubernetes client has been closed.
